In [6]:
# フォルダ内のpairのファイルを用いてCP, monoa, monob, formchkのshファイルを作成します。

import os

# フォルダの指定
input_folder_path = "data_for_cluster/C1/"

out_dir = input_folder_path.replace("data", "result")
os.makedirs(out_dir, exist_ok=True)
sh_name = input_folder_path.split("/")[-2]
out_sh_file = out_dir + sh_name + ".sh"

# 新しい5行目の内容
new_line_5 = '#P TEST b3lyp/6-311G** Empiricaldispersion=gd3 counterpoise=2\n'
new_line_9 = "0 1 0 1 0 1\n"
start_keyword = "X   1   r4   2   a4   3   d4"  # セクションの開始を示すキーワード
end_keyword = "Constants:"    # セクションの終了を示すキーワード
new_line_5_mono = "#P TEST b3lyp/6-311G** nosymm\n"



def process_file_CP(input_path, output_path):
    # Read the input file
    with open(input_path, 'r') as file:
        lines = file.readlines()
    
    # 5行目を置き換える
    lines[4] = new_line_5
    # 0 1 0 1 0 1に書き換え
    lines[8] = new_line_9
    
    start_line = end_line = None
    for i, line in enumerate(lines):
        if start_keyword in line:
            start_line = i
        elif end_keyword in line:
            end_line = i
            break  # 終了キーワードが見つかったらループを抜ける

#     HとMの書き換え
    if start_line is not None and end_line is not None:
        for i in range(start_line, end_line + 1):
            if lines[i].strip().endswith('H'):
                lines[i] = lines[i].rstrip('H\n') + '1\n'
            elif lines[i].strip().endswith('M'):
                lines[i] = lines[i].rstrip('M\n') + '2\n'

    # Write the processed content to the output file
    with open(output_path, 'w') as file:
        file.writelines(lines)

        
def process_file_monoa(input_path, output_path):
    # Read the input file
    with open(input_path, 'r') as file:
        lines = file.readlines()
    
    lines[0] = new_line_0_monoa
    # 5行目を置き換える
    lines[4] = new_line_5_mono
    
    start_line = end_line = None
    for i, line in enumerate(lines):
        if start_keyword in line:
            start_line = i
        elif end_keyword in line:
            end_line = i
            break  # 終了キーワードが見つかったらループを抜ける    
                
    if start_line is not None and end_line is not None:
        new_lines = []
        for i in range(start_line, end_line - 1):
            if lines[i].strip().endswith('H'):
                lines[i] = lines[i].rstrip('H\n') + '1\n'
                new_lines.append(lines[i])
        lines[start_line + 1:end_line - 1] = new_lines

    with open(output_path, 'w') as file:
        file.writelines(lines)

def process_file_monob(input_path, output_path):
    # Read the input file
    with open(input_path, 'r') as file:
        lines = file.readlines()
    
    lines[0] = new_line_0_monob
    # 5行目を置き換える
    lines[4] = new_line_5_mono
    
    start_line = end_line = None
    for i, line in enumerate(lines):
        if start_keyword in line:
            start_line = i
        elif end_keyword in line:
            end_line = i
            break  # 終了キーワードが見つかったらループを抜ける    
                
    if start_line is not None and end_line is not None:
        new_lines = []
        for i in range(start_line, end_line - 1):
            if lines[i].strip().endswith('M'):
                lines[i] = lines[i].rstrip('M\n') + '1\n'
                new_lines.append(lines[i])
        lines[start_line + 1:end_line - 1] = new_lines

    with open(output_path, 'w') as file:
        file.writelines(lines)

# 重複行削除の関数
def remove_duplicates(file_path, new_file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    unique_lines = list(set(lines))
    unique_lines.sort()  # オプション: ユニークな行をソートして順序を保持する

    with open(new_file_path, 'w') as new_file:
        new_file.writelines(unique_lines)

#     以下はフォルダ内のループ処理    
        
# フォルダ内のファイルをまとめて実行する
with open(out_sh_file,"w") as f:
    f.write("#! /bin/sh" + "\n")

for filename in os.listdir(input_folder_path):
    if filename.endswith('.gjf'):  # ここでファイルの種類を指定
        
        input_path = os.path.join(input_folder_path, filename)
        sample = input_path.split("/")[-1].split("@")[0]
        pair_n = input_path.split("/")[-1].split("_")[-1].split(".gjf")[0]
        f_namae = input_path.split("/")[-1].split(".gjf")[0]
        
        new_line_0_monoa = "%chk=" + "monoa_" + sample + "_" + "pair_" + pair_n + "\n"
        new_line_0_monob = "%chk=" + "monob_" + sample + "_" + "pair_" + pair_n + "\n"


        output_path_CP = out_dir + f_namae + "_CP.gjf"
        output_path_monoa = out_dir + f_namae + "_a.gjf"
        output_path_monob = out_dir + f_namae + "_b.gjf"
        
        process_file_CP(input_path, output_path_CP)
        process_file_monoa(input_path, output_path_monoa)
        process_file_monob(input_path, output_path_monob)
        
#         shファイルを書き出す処理
        for filename in os.listdir(out_dir):
            filename_ex_gjf = filename.split(".gjf")[0]
            
            if filename_ex_gjf.strip().endswith('a'):
                chk_path = new_line_0_monoa.split("%chk=")[-1].replace("\n", "") + ".chk"
                with open(out_sh_file,"a") as f:
                    text = "formchk " + chk_path + " " + chk_path.replace("chk", "fchk") + "\n"
                    f.write(text)

            if filename_ex_gjf.strip().endswith('b'):
                chk_path = new_line_0_monob.split("%chk=")[-1].replace("\n", "") + ".chk"
                with open(out_sh_file,"a") as f:
                    text = "formchk " + chk_path + " " + chk_path.replace("chk", "fchk") + "\n"
                    f.write(text)


file_path = out_sh_file  # 重複を削除するファイルのパス
new_file_path = out_sh_file  # 重複を削除した内容を保存する新しいファイルのパス

remove_duplicates(file_path, new_file_path)        
